# Exploring Montreal's buroughs with geopandas

In [1]:
#pip install scikit-learn geopandas h3pandas h3~=3.0 -q

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import leafmap

In [3]:
boroughs = gpd.read_file('../data/raw/borough-shapes.json')
bike_paths = gpd.read_file('../data/raw/bikelane-infra.json')
bixi_stations = gpd.read_file('../data/curated/bixi_stations.geojson')

In [4]:
bike_counters = pd.read_csv('../data/raw/bike-counts-locations.csv')
bike_counters = gpd.GeoDataFrame(
    bike_counters, 
    geometry=gpd.points_from_xy(bike_counters['Longitude'], bike_counters['Latitude']),
    crs='EPSG:4326'
)
bike_counters = bike_counters.to_crs(epsg=3857)
bike_counters['geometry'] = bike_counters['geometry'].buffer(100)  # Adjust the buffer size as needed

In [5]:
# Make sure all data is in the same projection. 3857 is the projection used by Google Maps in meters.
boroughs = boroughs.to_crs(epsg=3857)
bike_paths = bike_paths.to_crs(epsg=3857)
bixi_stations = bixi_stations.to_crs(epsg=3857)
bike_counters = bike_counters.to_crs(epsg=3857)

## Analyze boroughs

In [6]:
# Analysis for each borough
def analyze_borough(borough_geometry):
    # Count features intersecting with borough
    num_bike_counters = bike_counters[bike_counters.intersects(borough_geometry)].shape[0]
    num_bixi_stations = bixi_stations[bixi_stations.intersects(borough_geometry)].shape[0]
    num_bike_paths = bike_paths[bike_paths.intersects(borough_geometry)].shape[0]
    # Clip bike paths to the borough geometry to avoid double counting
    clipped_bike_paths = gpd.clip(bike_paths, borough_geometry)
    length_bike_paths = clipped_bike_paths.length.sum()

    return num_bike_counters, num_bike_paths, length_bike_paths, num_bixi_stations

In [7]:
# Apply analysis to each borough
boroughs[['num_bike_counters', 'num_bike_paths', 'length_bike_paths', 'num_bixi_stations']] = boroughs.geometry.apply(analyze_borough).apply(
    lambda x: pd.Series(x)
)
boroughs[['NOM', 'num_bike_counters', 'num_bike_paths', 'length_bike_paths', 'num_bixi_stations']].head()

,NOM,num_bike_counters,num_bike_paths,length_bike_paths,num_bixi_stations
0,LaSalle,0.0,254.0,42124.237283,18.0
1,Dollard-des-Ormeaux,0.0,162.0,28392.638905,0.0
2,Côte-Saint-Luc,0.0,122.0,19358.129245,0.0
3,Villeray-Saint-Michel-Parc-Extension,6.0,371.0,69559.753351,90.0
4,Rosemont-La Petite-Patrie,11.0,613.0,119652.810411,148.0


In [8]:
# Normalize results (0 to 1 scale)
# scaler = MinMaxScaler()
# columns_to_normalize = ['num_bike_counters', 'num_bike_paths', 'length_bike_paths']
# boroughs[columns_to_normalize] = scaler.fit_transform(boroughs[columns_to_normalize])
# boroughs[['NOM', 'num_bike_counters', 'num_bike_paths', 'length_bike_paths',]].head()

In [9]:
# Rating boroughs
index_score = boroughs['length_bike_paths'] #+ boroughs['num_bike_counters'] + boroughs['num_bike_paths']

In [10]:
m = leafmap.Map(center=[45.5, -73.6], zoom=11)
m.add_basemap("Google Maps")
m.add_data(boroughs, column='length_bike_paths', scheme='Quantiles', cmap='Blues', legend_title='Total Bike Path Length')
m

Map(center=[45.5, -73.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [11]:
m.add_gdf(bike_paths, layer_name="Bike Lanes", style= {'color': 'purple', 'opacity': 0.5})

In [12]:
m2 = leafmap.Map(center=[45.5, -73.6], zoom=11)
m2.add_basemap("Google Maps")
m2.add_data(boroughs, column='num_bixi_stations', scheme='Quantiles', cmap='Reds', legend_title='Total Bixi Stations')
m2

Map(center=[45.5, -73.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [13]:
m2.add_gdf(bixi_stations, layer_name="Bixi Stations", style= {'color': 'blue', 'opacity': 0.5})

In [20]:
# Create a GeoDataFrame that represents the area of the city
city_geometry = gpd.GeoDataFrame(geometry=[Polygon(boroughs.unary_union)], crs='EPSG:3857')

m3 = leafmap.Map(center=[45.5, -73.6], zoom=11)
m3.add_basemap("Google Maps")
m3.add_gdf(city_geometry, layer_name="City Area", style={'color': 'black', 'opacity': 0.5})
m3

# Export city area to a GeoJSON file
city_geometry.to_file('../data/curated/city-area.geojson', driver='GeoJSON')

Map(center=[45.5, -73.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…